# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** Classification, because the target variable is a discrete variable. We are classifying the students in two groups, whether they will graduate or not.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


In [3]:
(float(len(student_data[student_data['passed'] == 'yes']))/len(student_data))*100

67.08860759493672

### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [5]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns)

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = (float(len(student_data[student_data['passed'] == 'yes']))/len(student_data))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [6]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [7]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [8]:
# TODO: Import any additional functionality you may need here
# train_test_split splits data into **random** train and test subsets
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


### Correlation between variables

In [9]:
from math import sqrt

def square(x):
    return x**2

#Compute value of 80th percentile to assess how strong independence (correlation of varibales are)
# Absolute value was considered so negative and positive values don't cancel each other out
X_all.corr(method='pearson').apply(square).apply(np.sqrt).quantile(.80).max()

0.1894483189935979

## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

1. _**Gaussian Naive Bayes**_:  

    **Example**: Naive Bayes is very popular in commercial and open-source anti-spam e-mail filters [(Metsis, et al., 2006)](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). In other words classifying if an email should be cosidered as spam or not. Many times a textual analysis is carried out and probabilities of certain words or headers are taken into account as atributes.
    
    **Advantages**: It's simple to implement and has a linear computational complexity (Metsis, et al., 2006). Basically you're just doing counts. It performs best when independence between attributes holds, as it is its basic assumption. Moreover it still performs well in practice, even if conditional indpendence of the variables doesn't hold, which is a huge advantage considering that's rarely the case in the real world.
    
    **Disadvantages**: One important disadvantage is that it can’t learn relationships between features, as it assumes independence between them. Another issue is that if you have no occurrences of a class label and a certain attribute value together (class="passed", travel_time="50") then the frequency-based probability estimate will be zero when the porbabilities of the variables are multiplied, even though in reality there might be instances left out of the data set, that have these occurrences of the class and a specific attribute.
    
    **Why good for problem**: Naive Bayes is one of the most effective, efficient and competitive machine learning algorithms for classification [(Zhang, 2004)](http://www.cs.unb.ca/~hzhang/publications/FLAIRS04ZhangH.pdf). Which is the problem we are dealing here with. Secondly given we are asked to this this as optimally as possible, the low computational requirements make it a good fit for this case too. Finally as seen in the section "Correlation between Variables", I've computed the 80th percentile of all correlations between variables, where the maximum value was 0.19. This indicates that the majority of the variables have low correlation (hence low indpendence), which is a good indication for using naive Bayes.
    
    
2. _**Decision Trees**_:  

    **Example**: Automated star/galaxy classification [(Weir, et al., 1995)](https://www.researchgate.net/profile/George_Djorgovski/publication/234223141_Automated_StarGalaxy_Classification_for_Digitized_Poss-II/links/554bc71c0cf29752ee7eba39.pdf). Decision treess were used here to classify objects in the sky as stars or distant galaxies depending on radius of the object, elipticity and others. 
    
    **Advantages**: Complex data can be easily presented in a decision tree and decision trees do not require any assumptions of linearity in the data. Also if the data is not overlapping much (i.e. the classes are very distinct) they can perfomr very well even with a low depth.
    
    **Disadvantages**:  DTs have is they are high variance classifiers i.e. the DT learnt is sensitive to the precise layout of points and, if you have less data, can fit to noise. In other words DTs don't perform well when there is not much data or features, so they tend to overfit and fail to generalize on new data.
    
    **Why good for problem**: First of all they are well suited for a classification problem, like the one we are approaching. DTs will also easily tell us which are the most important attributes that separate the two groups, which are the first nodes the tree seperates by, calculated with the information gain. This will make it easier to understand for scholars, who are not familiar with machine leanring algorithms, where action can be taken and will have the highest impact. Moreover we don't need to normalize data which will lead to use less resources (as asked) by the contractor.
    
    
3. _**Support Vector Machines**_:

    **Example**: SVM is used in this example as a classificattion method to see whether cancer tissues have distinctive signatures of gene expression over normal tissues. ([See link)](http://link.springer.com/article/10.1023/A:1012487302797) 
    
    **Advantages**: Work well when there is a clear margin of separation between the classes. Moreover this separation doesn't necessarily have to be linear, as SVM supports kernels, so you can model even non-linear relations. Finally they maximize the margin, which makes them more robust and less likely to high have variance.
    
    **Disadvantages**: They don't work well where there is a lot of noise in the data and when the target classes are very overlapping. Moreover, if the datasets are large their computation is cubic as a function of the size of the dataset, which makes it computationally expensive.
    
    **Why good for problem**: Again it's a classification problem, so SVM are a good way to appraoch it. The dataset is not large so we won't the cubuc cumutational problem.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [66]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [78]:
# TODO: Import the three supervised learning models from sklearn
from sklearn import svm
from sklearn import tree 
from sklearn.naive_bayes import GaussianNB

# TODO: Initialize the three models
clf_A = svm.SVC(random_state=10)
clf_B = tree.DecisionTreeClassifier(random_state=10)
clf_C = GaussianNB()

# TODO: Set up the training set sizes
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train.head(300)
y_train_300 = y_train.head(300)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

In [112]:
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6441.


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

** Classifer 1 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0055s             |    0.0014s             |        0.8777    |       0.7746    |
| 200               |     0.0042s             |    0.0024s             |        0.8679    |       0.7815    |
| 300               |     0.0092s             |    0.0069s             |        0.8761    |0.7838 |

** Classifer 2 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0115s          |  0.0004s               |     1.0000       |       0.6441    |
| 200               |     0.0020s             |  0.0002s               |     1.0000       |   0.7244        |
| 300               |   0.0027s                 |  0.0002s                      |     1.0000     |     0.6066     |

** Classifer 3 - Naive Bayes Gaussian**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0016s             |      0.0003s        |         0.8467   |    0.8029             |
| 200               |        0.0017s        |           0.0003s       |         0.8406     |          0.7244   |
| 300               |           0.0016s     |          0.0005s          |        0.8083       |      0.7634   |

### Exploring the metrics

In [115]:
#Average SVM
print 'SVM Mean:' + str(float(0.7746+0.7815+0.7838)/3) + ' SVM Std ' + str(np.std([0.7746,0.7815,0.7838]))

#Average DT
print 'DT Mean:' + str(float(0.6441+0.7244+0.6066)/3) + ' DT Std ' + str(np.std([0.6441,0.7244,0.6066]))

#Average NB
print 'NB Mean:' + str(float(0.8029+0.7244+0.7634)/3) + ' NB Std ' + str(np.std([0.8029,0.7244,0.7634]))

SVM Mean:0.779966666667 SVM Std 0.00390924829375
DT Mean:0.658366666667 DT Std 0.0491383308178
NB Mean:0.763566666667 NB Std 0.032047707493


In [124]:
list(y_test).count('yes')

60

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Overall I would choose Support Vector Machines as our final model. It not only achieved the highest average f1 score (as seen above) but also has the smallest standard devation in terms of its scores. This indicates good robustness for unseen data. Moreover it's seems it is not as overfit as the Decision tree, the f1 score is not as close to one. 

Finally SVM seems to improve when we use more data to train it, which for instance Naive Bayes doesn't. When we have more training data, we are more likely to represent the whole population with that data, hence more likely to make a better estimation with the test data (which in this case has almost the same distribution 35 'no' and 60 'yes'). So we would expect a better estimation with more data.

The only caveat of SVm is that is uses more resources than both of the other algorithms, but since we only have this data we are dealing with very low usage of resources, so I'd still use the f1 score as the main argument for chosing SVM.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** SVM tries to create a clear margin os separation between the students who passed and the ones who didn't. Imagine the students who passed are standing standing on a basketball court on one side and the ones who didn't on the other. Their distance to the middle is their average gpa, so users who passed by a few points are closest to the middle and those with high grades are furthest form the middle. On the other side, the ones who didn't pass by a low margin are closest to the middle and the onse with the worst grades are further away. 

SVM looks at both groups and tries to draw this imaginary line in such a way that the distance from the closest student from each side is the farthest away possible (It doens't necessarly has to be the middle line of the court in this case). After drawing the line that maximizes both distances, it will learn that all new students who land on the side os the one who passed are going to be labeld as passed and the same with the other side.

In our example, we have more than one variable, but that is exactly what our model does in a multidimensional space.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [151]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'kernel' : ['rbf','poly',], 'C' : [1,10,100,1000,10000], 'gamma' : [0.1,0.2] }

# TODO: Initialize the classifier
clf = svm.SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.1,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Made predictions in 0.0056 seconds.
Tuned model has a training F1 score of 0.9762.
Made predictions in 0.0019 seconds.
Tuned model has a testing F1 score of 0.7895.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.